In [1]:
!pip install pandas
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [105]:
import pandas as pd
import re
import sqlite3

def read_derived_rules(query):
    conn = sqlite3.connect('T3.db')
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    conn.close()
    return data

queries = {
    'components': 'SELECT Component FROM T3_table',
    'project_name': 'SELECT [Project name] FROM T3_table',
    'version': 'SELECT Version FROM T3_table',
    'date': 'SELECT Date FROM T3_table',
    'owner': 'SELECT Owner FROM T3_table',
    'OD_rules': 'SELECT [OD rules] FROM T3_table'
}

components = [row[0] for row in read_derived_rules(queries['components'])]
project_name = [row[0] for row in read_derived_rules(queries['project_name'])]
version = [row[0] for row in read_derived_rules(queries['version'])]
date = [row[0] for row in read_derived_rules(queries['date'])]
owner = [row[0] for row in read_derived_rules(queries['owner'])]
OD_rules = [row[0] for row in read_derived_rules(queries['OD_rules']) if row[0] is not None]

rules = [row[0] for row in read_derived_rules('SELECT rules FROM T3_table WHERE Type="Derive"') if row[0] is not None]
newrules = [line.split(']')[0] for line in rules]

pattern = r'([✔✘])\s*(.*)'
data = []
section_id = 1

for section in OD_rules:
    items = re.split(r'\s*\|\|\s*|\s*&&\s*|\s*\(\s*|\s*\)\s*', section)
    section_data = []
    for item in items:
        match = re.match(pattern, item)
        if match:
            status, name = match.groups()
            section_data.append((section_id, status, name))
    if section_data:
        data.extend(section_data)
    section_id += 1

df = pd.DataFrame(data, columns=['Section_id', 'Status', 'name'])
df['name'] = df['name'].str.replace(r'\bOnly\b.*', '', regex=True).str.strip()
df['name'] = df['name'].str.replace(r'\bSBB\w*\b', '', regex=True).str.strip()
df['Section_id'] = pd.factorize(df['Section_id'])[0] + 1

print(df)


       Section_id Status                                               name
0               1      ✔                            TW 9225 Rear System fan
1               1      ✔                   9225 Rear Photocataly fan for TW
2               1      ✔                           TW 8025 Front system fan
3               1      ✔                       RTX3050 4GB G6 128b DVI+H+DP
4               1      ✔                         RTX3060 12GB G6 192b H+3DP
...           ...    ...                                                ...
22122        7719      ✔                              Windows 10 Home 64 EM
22123        7719      ✔                              Windows 11 Home 64 EM
22124        7719      ✔                             F MB Dali FP5 3020E MP
22125        7720      ✔  USB Calliope Keyboard Black US English 103P  103P
22126        7721      ✔                              USB Fingerprint Mouse

[22127 rows x 3 columns]


In [106]:
import sqlite3 
from py2neo import Graph, Node, Relationship
g = Graph('neo4j://localhost:7687', auth = ('neo4j', 'ys1203303'))
g.run("MATCH (n) DETACH DELETE n") 
for i in range(7720):
    node1 = Node(
            "parent",
            Name = newrules[i],
            Component = components[i],
            originalRule = rules[i],
            Type = "Derive",
            ProjectName = project_name[i],
            Version = version[i],
            owner = owner[i], 
            date = date[i],
            ruleIndex = OD_rules[i])
    g.create(node1) 
    section_df = df[df['Section_id'] == i + 1] 
    for s, ss in section_df.iterrows():
        node2 = Node(
            "son", 
            Name = ss['name'],
            Component = components[i],
            originalRule = rules[i],
            Type = "Derive",
            ProjectName = project_name[i],
            Version = version[i],
            owner = owner[i], 
            date = date[i],
            ruleIndex = OD_rules[i])
        g.create(node2)
        if ss['Status'] == '✔':
            relationship = Relationship(node2, "should", node1) 
        else:
            relationship = Relationship(node2, "should not", node1) 
        g.create(relationship) 